In [1]:
import torch
import copy, os
import logging
import numpy as np
import out_manager as om
from config import Config
import torch.nn.functional as F
from torch_geometric.utils import negative_sampling
from sklearn.metrics import roc_auc_score, average_precision_score
from model.score_gnn import scoregnn_dict

In [2]:
config = Config()
ModelClass = scoregnn_dict[config.scoregnn.gnn_type]
out_dir = om.get_out_dir(config)
log_path = os.path.join(out_dir, "scoregnn_log.txt")
om.setup_logging(log_path)
seed = config.seed
torch.manual_seed(seed)
np.random.seed(seed)
device = config.device

In [3]:
def train(model, data, optimizer, predictor):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)

    # #每次手动采样负样本，效果更好
    # neg_edge_index = negative_sampling(
    #     edge_index = data.edge_index,
    #     num_nodes = data.num_nodes,
    #     num_neg_sample = data.edge_index.size(1)
    # )

    # edge_label_index = torch.cat([
    #     data.edge_label_index, neg_edge_index
    #     ], dim=-1)
    # edge_label = torch.cat([
    #     data.edge_label, data.edge_label.new_zeros(neg_edge_index.size(1))
    #     ],dim = 0)

    edge_label_index = torch.cat([
        data.pos_edge_label_index, data.neg_edge_label_index
    ], dim=1)
    edge_label = torch.cat([
        data.pos_edge_label, data.neg_edge_label
    ], dim=0)
    
    score = predictor(out, edge_label_index)
    loss = F.binary_cross_entropy_with_logits(score, edge_label)

    loss.backward()
    optimizer.step()
    return loss.item()
    

In [4]:
@torch.no_grad()
def test(model, data, predictor):
    model.eval()

    edge_label_index = torch.cat([
        data.pos_edge_label_index, data.neg_edge_label_index
    ], dim=1)
    edge_label = torch.cat([
        data.pos_edge_label, data.neg_edge_label
    ], dim=0)

    out = model(data.x, data.edge_index)
    score = predictor(out, edge_label_index).cpu().numpy()
    auc = roc_auc_score(edge_label.cpu().numpy(), score)
    ap = average_precision_score(edge_label.cpu().numpy(), score)
    return auc, ap

In [5]:
train_data = torch.load(f'./data/{config.dataset}/split/train_data.pt').to(device)
val_data = torch.load(f'./data/{config.dataset}/split/val_data.pt').to(device)
test_data = torch.load(f'./data/{config.dataset}/split/test_data.pt').to(device)

model = ModelClass(config.data_init_num_features, hidden_dim = config.scoregnn.hidden_dim, 
                 output_dim = config.scoregnn.output_dim , num_layers = config.scoregnn.num_layers, 
                 dropout = config.scoregnn.dropout).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config.scoregnn.lr)

final_model = final_predictor = None
best_val_auc = final_test_auc = final_test_ap = 0

model.reset_parameters()

for epoch in range(1, 1 + config.scoregnn.epochs):
    loss = train(model, train_data, optimizer, config.scoregnn.predictor)
    train_auc, train_ap = test(model, train_data, config.scoregnn.predictor)
    val_auc, val_ap = test(model, val_data, config.scoregnn.predictor)
    test_auc, test_ap = test(model, test_data, config.scoregnn.predictor)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
        final_test_ap = test_ap
        final_model = copy.deepcopy(model)
        final_predictor = copy.deepcopy(config.scoregnn.predictor)

    logging.info(f'Epoch: {epoch:03d}, Loss: {loss:.4f} '
             f'Train_AUC: {train_auc:.4f}, Train_AP: {train_ap:.4f} '
             f'Val_AUC: {val_auc:.4f}, Val_AP: {val_ap:.4f} '
             f'Test_AUC: {test_auc:.4f}, Test_AP: {test_ap:.4f}')
    
logging.info(f'Final Test AUC: {final_test_auc:.4f}, AP: {final_test_ap:.4f}')

Epoch: 001, Loss: 0.6881 Train_AUC: 0.6967, Train_AP: 0.7162 Val_AUC: 0.6828, Val_AP: 0.7125 Test_AUC: 0.6734, Test_AP: 0.7084
Epoch: 002, Loss: 0.6005 Train_AUC: 0.7696, Train_AP: 0.7717 Val_AUC: 0.7331, Val_AP: 0.7520 Test_AUC: 0.7361, Test_AP: 0.7552
Epoch: 003, Loss: 0.5690 Train_AUC: 0.7175, Train_AP: 0.6831 Val_AUC: 0.7007, Val_AP: 0.6685 Test_AUC: 0.6985, Test_AP: 0.6639
Epoch: 004, Loss: 0.5247 Train_AUC: 0.5682, Train_AP: 0.5842 Val_AUC: 0.5628, Val_AP: 0.5798 Test_AUC: 0.5540, Test_AP: 0.5688
Epoch: 005, Loss: 0.4885 Train_AUC: 0.5277, Train_AP: 0.5710 Val_AUC: 0.5247, Val_AP: 0.5685 Test_AUC: 0.5188, Test_AP: 0.5585
Epoch: 006, Loss: 0.4600 Train_AUC: 0.6202, Train_AP: 0.6113 Val_AUC: 0.6215, Val_AP: 0.6096 Test_AUC: 0.6103, Test_AP: 0.5995
Epoch: 007, Loss: 0.4408 Train_AUC: 0.7689, Train_AP: 0.7604 Val_AUC: 0.7650, Val_AP: 0.7631 Test_AUC: 0.7559, Test_AP: 0.7484
Epoch: 008, Loss: 0.4314 Train_AUC: 0.8028, Train_AP: 0.7933 Val_AUC: 0.7900, Val_AP: 0.7898 Test_AUC: 0.7880, 

In [6]:
torch.save({
    'model': final_model.state_dict(),
    'predictor': final_predictor.state_dict()
}, './model/scoregnn.pth')